In [19]:
import numpy as np
import scipy.stats as st


Contingency_Table = np.array([[4,4,2],[2,6,0],[0,2,0]])
confidence_level = 0.95

def calculate_p_value_from_z_score(score):
    p_value = st.t.sf((abs(score)), 100000) * 2
    return min(float(p_value), 0.99999)

# Note that we used the CI irrCAC used the t distribution while we used the Z distribution as most packages do

def scotts_pi(Contingency_Table, confidence_level):
    Sample_Size = np.sum(Contingency_Table)
    Number_Of_Levels = Contingency_Table.shape[1]

    Weights = np.eye(Number_Of_Levels)
    Percentages_of_Agreement = np.sum(Weights * Contingency_Table / Sample_Size)
    Sum_of_Coloumns = np.sum(Contingency_Table, axis=0) / Sample_Size
    Sum_of_Rows = np.transpose(np.sum(Contingency_Table, axis=1) / Sample_Size)
    Joint_Distribution = (Sum_of_Coloumns + Sum_of_Rows) / 2
    Percentage_Of_Disagreement = np.sum(Weights * np.outer(Joint_Distribution, Joint_Distribution))
    Scotts_Pi = (Percentages_of_Agreement - Percentage_Of_Disagreement) / (1 - Percentage_Of_Disagreement)

    # Calcaulate the Varaince
    Probability_Table = Contingency_Table / Sample_Size
    Probability_Rows = np.dot(Weights, Sum_of_Rows)
    Probability_Coloumns = np.dot(np.transpose(Weights), Sum_of_Coloumns)
    Joint_Probabilities = (Probability_Rows + Probability_Coloumns) / 2
    Term1 = sum(Probability_Table[k, l] * (Weights[k, l] - (1 - Scotts_Pi) * (Joint_Probabilities[k] + Joint_Probabilities[l]))**2 for k in range(Number_Of_Levels) for l in range(Number_Of_Levels))
    Standart_Error_Scotts_Pi = np.sqrt((1 / (Sample_Size * (1 - Percentage_Of_Disagreement)**2)) * (Term1 - (Percentages_of_Agreement - 2 * (1 - Scotts_Pi) * Percentage_Of_Disagreement)**2))

    # Significance and confidence Intervals
    Statistic = Scotts_Pi / Standart_Error_Scotts_Pi
    p_value = calculate_p_value_from_z_score(Statistic)

    # Confidence Interval
    zcrit = st.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_Confidence_Interval_Scotts_Pi = Scotts_Pi - Standart_Error_Scotts_Pi*zcrit
    Upper_Confidence_Interval_Kappa_Scotts_Pi = Scotts_Pi + Standart_Error_Scotts_Pi*zcrit

    results = {}

    results["Scott's Phi"]= Scotts_Pi
    results["Z-Statistic"]= Statistic
    results["p_value"] = p_value
    results["Standart Error Scott's Phi"] = Standart_Error_Scotts_Pi
    results["Confidence Intervals Kappa"] = f"({round(Lower_Confidence_Interval_Scotts_Pi, 4)}, {round(Upper_Confidence_Interval_Kappa_Scotts_Pi, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results

scotts_pi(Contingency_Table, confidence_level)




{"Scott's Phi": 0.13793103448275856,
 'Z-Statistic': 0.7986952322845514,
 'p_value': 0.42446904883878955,
 "Standart Error Scott's Phi": 0.17269545241709647,
 'Confidence Intervals Kappa': '(-0.2005, 0.4764)'}